# CPDCTL Demo - Python script lifecycle

CPDCTL is a command-line interface (CLI) you can use to manage the lifecycle of notebooks. By using the notebook CLI, you can automate the flow for creating notebooks and running notebook jobs, moving notebooks between projects in Watson Studio, and adding custom libraries to notebook runtime environments.   

This notebook begins by showing you how to install and configure CPDCTL and is then split up into three sections with examples of how to use the commands to:

- Promote python script from the project to the space
- Promote python script to the QA space
- Download the script from the QA space and upload it to the production space

## Table of Contents

[1. Install and Configure CPDCTL](#part1)
- [1.1 Installing the latest version of CPDCTL](#part1.1)
- [1.2 Adding CPD cluster configuration settings](#part1.2)

[2. Demo 1: Promote python script from the project to the space](#part2)
- [2.1 Creating a notebook asset](#part2.1)
- [2.2 Running a deployment job](#part2.2)

[3. Demo 2: Promote python script to the QA space](#part3)
- [3.1 Export all asssets from the source (DEV) space](#part3.1)
- [3.2 Create QA space and import assets there](#part3.2)
- [3.3 Run batch deployment job in the QA space](#part3.3)

[4. Demo 3: Download the script from the QA space and upload it to the production space](#part4)
- [4.1 Downloading a script](#part4.1)
- [4.2 Creating the new script in the production space](#part4.2)
- [4.2 Updating the existing script deployment](#part4.3)

[5. Cleanup](#part5)

## Before you begin
Import the following libraries:

In [171]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from datetime import datetime
from IPython.core.display import display, HTML

##  1. Installing and configurating CPDCTL <a class="anchor" id="part1"></a>

### 1.1 Installing the latest version of CPDCTL <a class="anchor" id="part1.1"></a>

To use the notebook and environment CLI commands, you need to install CPDCTL. Download the binary from the [CPDCTL GitHub respository](https://github.com/IBM/cpdctl/releases).

Download the binary and then display the version number:

In [219]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPDCONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))
display(HTML("<style>.container { width:90% !important; }</style>"))

In [24]:
%%capture

%env PATH={CWD}:{PATH}
%env CPDCONFIG={CPDCONFIG}

In [25]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPDCONFIG and os.path.exists(CPDCONFIG):
    os.remove(CPDCONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.0.22


### 1.2  Adding CPD cluster configuration settings <a class="anchor" id="part1.2"></a>

Before you can use CPDCTL, you need to add configuration settings. You only need to configure these settings once for the same CPD user and cluster. Begin by entering your IBM Cloud Pak for Data (CPD) credentials and the URL to the CPD cluster:

In [ ]:
CPD_USER_NAME = #'YOUR CPD user name'
CPD_USER_PASSWORD = #'YOUR CPD user password'
CPD_URL = #'YOUR CPD CLUSTER URL'

Add "cpd_user" user to the cpdctl configuration:

In [27]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}

Add "cpd" cluster to the cpdctl configuration:

In [28]:
! cpdctl config profile set cpd --url {CPD_URL}

Add "cpd" context to the cpdctl configuration:

In [29]:
! cpdctl config context set cpd --profile cpd --user cpd_user

List available contexts:

In [30]:
! cpdctl config context list

Name   Profile   User       Current   
cpd    cpd       cpd_user   *   


Switch the current context:

In [31]:
! cpdctl config context use cpd

Switched to context "cpd".


List available projects in context:

In [32]:
! cpdctl project list

...
ID                                     Name               Created                    Description   Tags   
0cabb425-56d9-48fc-a178-f30d22737778   git-demo-project   2021-02-19T13:47:11.811Z                 []   
7fb76cf7-25be-435d-818e-bd6e9b5254f5   cpdctl-demo        2021-01-29T08:01:23.363Z                 []   


Choose a project in which you will work:

In [33]:
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[0]" --raw-output
project_id = result.s
print("project id: {}".format(project_id))

# You can also specify your project id directly:
# project_id = "Your project ID"

project id: 0cabb425-56d9-48fc-a178-f30d22737778


## 2. Demo 1: Promote python script from the project to the space  <a class="anchor" id="part2"></a>

Before starting with this section, please ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a python script created with JupyterLab in Watson Studio and you would like to run the code on a CPD cluster. This section shows how to promote a script asset from a project to a space and run a job on a CPD cluster. 

### 2.1 Promote script asset to the space<a class="anchor" id="part2.1"></a>

List all the script assets in your project, filter them by their display name and get the ID of the script:

In [220]:
! cpdctl asset search --project-id {project_id} --type-name script --query "*:*"

...
ID                                     Name               Created                    Description   Type     State       Tags   Size   
14766a5b-b842-4f19-b2d0-690b64e46d25   batch_job_script   2021-02-19T22:08:45.000Z                 script   available   []     4183   


In [211]:
script_name = "batch_job_script"
query = "asset.name:{}".format(script_name)
jmes_query = "results[0].metadata.asset_id"

In [212]:
result = ! cpdctl asset search --project-id {project_id} --query {query} --type-name script --output json --jmes-query "{jmes_query}" --raw-output
script_id = result.s
print("script id: {}".format(script_id))

script id: 14766a5b-b842-4f19-b2d0-690b64e46d25


List all spaces

In [213]:
! cpdctl space list

...
ID                                     Name                            Created                    State    Tags   
d9bfa660-1be7-46ab-aa53-e9010a634bba   cpdctl-demo-space               2021-01-29T08:56:07.389Z   active   []   
691dfe59-ffed-4241-a2a1-9c49be5aaf02   cpdctl-qa-space-for-scripts     2021-02-22T06:39:52.763Z   active   []   
6c205951-1c61-49b9-b46d-e5e199492775   cpdctl-prod-space-for-scripts   2021-02-22T07:45:34.787Z   active   []   


Select the 'cpdctl-demo-space' space

In [215]:
dev_space_name = 'cpdctl-demo-space'
jmes_query = "resources[?entity.name == '{}'] | [0].metadata.id".format(dev_space_name)
result = ! cpdctl space list --output json --jmes-query "{jmes_query}" --raw-output
space_id = result.s
print('Space ID: {}'.format(space_id))

Space ID: d9bfa660-1be7-46ab-aa53-e9010a634bba


Select script asset for promotion and provide expected name and metadata:

In [43]:
import json

promote = {
    "mode": 0,
    "space_id": space_id,
    "metadata": {
        "name": "batch_job_script.py",
        "tags": ["cpdctl-demo", "promoted-asset-{}]".format(script_id)]
    }
}
promote_json = json.dumps(promote)

! cpdctl asset promote --project-id {project_id} --asset-id {script_id} --request-body '{promote_json}'

...
OK


List assets in the space

In [222]:
! cpdctl asset search --space-id {space_id} --type-name script --query "*:*"

...
ID                                     Name                  Created                    Description   Type     State       Tags                                                                 Size   
d2f732ec-fc38-4d6e-90fb-6b9739b8fdba   batch_job_script.py   2021-02-19T22:26:10.000Z                 script   available   [cpdctl-demo promoted-asset-14766a5b-b842-4f19-b2d0-690b64e46d25]]   4183   


Select the promoted script

In [48]:
query = 'asset.name:batch_job_script.py'
jmes_query = "results[0].metadata.asset_id"
result = ! cpdctl asset search --space-id {space_id} --query {query} --type-name script --output json --jmes-query "{jmes_query}" --raw-output
promoted_script_id = result.s
print("promoted script id: {}".format(promoted_script_id))

promoted script id: d2f732ec-fc38-4d6e-90fb-6b9739b8fdba


List software specifications

In [80]:
software_specification_name = "default_py3.7"
jmes_query = "resources[0].metadata.asset_id"
result = ! cpdctl environment software-specification list --space-id {space_id} --name '{software_specification_name}' --output json --jmes-query '{jmes_query}' --raw-output
software_specification_id = result.s
print("software specification id: {}".format(software_specification_id))

software specification id: e4429883-c883-42b6-87a8-f419d64088cd


Set python script's software spececification

In [82]:
software_spec = {
    "base_id": "{}".format(software_specification_id),
    "name": software_specification_name
}

patch = [{
    "op": "replace",
    "path": "/software_spec",
    "value": software_spec
}]
patch_json = json.dumps(patch)

! cpdctl asset attribute update --space-id {space_id} --asset-id {promoted_script_id} --attribute-key script  --json-patch '{patch_json}'

...
OK


### 2.2 Run batch deployment job <a class="anchor" id="part2.2"></a>

Create batch deployment:

In [89]:
asset = {
    'id': promoted_script_id
}
asset_json = json.dumps(asset)

hardware_spec = {
    'name': 'S'
}
hardware_spec_json = json.dumps(hardware_spec)

batch_json = '{}'

deployment_name = 'script_batch_deployment'

In [90]:
result = ! cpdctl ml deployment create --space-id {space_id} --name '{deployment_name}' --asset '{asset_json}' --hardware-spec '{hardware_spec_json}' --batch '{batch_json}' --output json -j "metadata.id" --raw-output
deployment_id = result.s
print("deployment id: {}".format(deployment_id))

deployment id: 94c2de39-51ab-40a1-b717-92a7991c420e


Create a deployment job

In [101]:
deployment_job_name = 'script_batch_deployment_job'

deployment = {
    'id': deployment_id
}
deployment_json = json.dumps(deployment)

scoring = {
    "input_data_references": [
      {
        "type": "data_asset",
        "connection": {},
        "location": {
          "href": "/v2/assets/783d8fc5-ae2c-47d0-a311-f8890dfa1ce0?space_id=d9bfa660-1be7-46ab-aa53-e9010a634bba"
        }
      }
    ],
    "output_data_reference": {
      "type": "data_asset",
      "connection": {},
      "location": {
        "href": "/v2/assets/c7de9cb4-0ec5-41fa-8b94-3e89eb2cb795?space_id=d9bfa660-1be7-46ab-aa53-e9010a634bba"
      }
    }
}
scoring_json = json.dumps(scoring)

In [103]:
result = ! cpdctl ml deployment-job create --space-id {space_id} --name '{deployment_job_name}' --deployment '{deployment_json}' --scoring '{scoring_json}' --output json
deployment_job = json.loads(result.s)
print(json.dumps(deployment_job, indent=2))
job_id = deployment_job['entity']['platform_job']['job_id']
run_id = deployment_job['entity']['platform_job']['run_id']

{
  "entity": {
    "deployment": {
      "id": "94c2de39-51ab-40a1-b717-92a7991c420e"
    },
    "platform_job": {
      "job_id": "6b2d1087-2728-4d72-b5de-0cf0dd633912",
      "run_id": "8af92e6d-8cc4-42ad-8143-f74bad639ff7"
    },
    "scoring": {
      "input_data_references": [
        {
          "connection": {},
          "location": {
            "href": "/v2/assets/783d8fc5-ae2c-47d0-a311-f8890dfa1ce0?space_id=d9bfa660-1be7-46ab-aa53-e9010a634bba"
          },
          "type": "data_asset"
        }
      ],
      "output_data_reference": {
        "connection": {},
        "location": {
          "href": "/v2/assets/c7de9cb4-0ec5-41fa-8b94-3e89eb2cb795?space_id=d9bfa660-1be7-46ab-aa53-e9010a634bba"
        },
        "type": "data_asset"
      },
      "status": {
        "state": "queued"
      }
    }
  },
  "metadata": {
    "created_at": "2021-02-21T21:21:25.338Z",
    "id": "a6da3389-59b4-4f7f-bf34-b037f6721ba5",
    "name": "script_batch_deployment_job",
    "space_id

Wait for job completion

In [106]:
! cpdctl job run wait --job-id {job_id} --run-id {run_id} --space-id {space_id}

...
                  
ID:            8af92e6d-8cc4-42ad-8143-f74bad639ff7   
Name:          job run   
Created:       2021-02-21T21:21:25Z   
Description:      
State:         Completed   
Tags:          []   


You can see the batch deployment log:

In [107]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --space-id {space_id}

...
total_count   results   
36            {   
36              "deployment": {   
36                "id": "94c2de39-51ab-40a1-b717-92a7991c420e"   
36              },   
36              "platform_job": {   
36                "job_id": "6b2d1087-2728-4d72-b5de-0cf0dd633912",   
36                "run_id": "8af92e6d-8cc4-42ad-8143-f74bad639ff7"   
36              },   
36              "scoring": {   
36                "input_data_references": [   
36                  {   
36                    "connection": {},   
36                    "location": {   
36                      "href": "/v2/assets/783d8fc5-ae2c-47d0-a311-f8890dfa1ce0?space_id=d9bfa660-1be7-46ab-aa53-e9010a634bba"   
36                    },   
36                    "type": "data_asset"   
36                  }   
36                ],   
36                "output_data_reference": {   
36                  "connection": {},   
36                  "location": {   
36                    "href": "/v2/assets/c7de9cb4-0ec5-41fa-8

## 3. Demo 2: Promoting python script to the QA space <a class="anchor" id="part3"></a>

Before starting with this section, please ensure that you have run the cells in [Section 1](#part1) and [Section 2](#part2).

Suppose you have a Python script (.py) on your local system and you would like to run the code in the script as a job on a CPD cluster. This section shows you how to create a Python script asset and run a job on a CPD cluster.

### 3.1 Export all asssets from the source (DEV) space<a class="anchor" id="part3.1"></a>

List all assets in the source (DEV) space

In [223]:
! cpdctl asset search --space-id {space_id} --query '*:*' --type-name asset

...
ID                                     Name                                                      Created                    Description   Type         State       Tags                                                                 Size   
384ddcde-a7fe-4652-baa9-b41c40b08284   boston-house-prices-prediction-model                      2021-01-29T09:27:11.000Z                 wml_model    available   []                                                                   839   
d59ff8a1-e58b-4a6c-882c-a6ebfe3c7a70   batch-job-outputs-connection                              2021-02-19T16:53:10.000Z                 connection   available   []                                                                   0   
6b2d1087-2728-4d72-b5de-0cf0dd633912   WML-Deployment-Job-94c2de39-51ab-40a1-b717-92a7991c420e   2021-02-21T21:19:25.000Z                 job          available   []                                                                   0   
783d8fc5-ae2c-47d0-a311-f8890dfa1ce0   car_

Export all assets from the source (DEV) space

In [110]:
EXPORT = {
    'all_assets': True
}
EXPORT_JSON = json.dumps(EXPORT)
result = ! cpdctl asset export start --space-id {space_id} --assets '{EXPORT_JSON}' --name dev-space-all-assets --output json --jmes-query "metadata.id"
export_id = result.s
print("The new export with ID: {}".format(export_id))

The new export with ID: "3916bbb2-9106-4125-b7a7-7368f36df48d"


In [112]:
! cpdctl asset export get --space-id {space_id} --export-id {export_id}

...
              
ID:        3916bbb2-9106-4125-b7a7-7368f36df48d   
Name:      dev-space-all-assets   
Created:   2021-02-22T06:35:18.568Z   
State:     completed   


In [114]:
dev_space_archive_path = './dev-space-assets.zip'
! cpdctl asset export download --space-id {space_id} --export-id {export_id} --output-file {dev_space_archive_path}

...
OK
Output written to dev-space-assets.zip


In [118]:
! ls -al {dev_space_archive_path}

-rw-r--r--  1 rafalbigaj  staff  58590 Feb 22 07:37 ./dev-space-assets.zip


### 3.2 Create QA space and import assets there<a class="anchor" id="part3.2"></a>

Create a new QA space

In [116]:
qa_space_name = 'cpdctl-qa-space-for-scripts'
result = ! cpdctl space create --name '{qa_space_name}' --output json --jmes-query "metadata.id" --raw-output
qa_space_id = result.s
print("The new '{}' space ID is: {}".format(qa_space_name, qa_space_id))

The new 'cpdctl-qa-space-for-scripts' space ID is: 691dfe59-ffed-4241-a2a1-9c49be5aaf02


Import assets from the exported archive into QA space

In [119]:
result = ! cpdctl asset import start --space-id {qa_space_id} --import-file {dev_space_archive_path} --output json --jmes-query "metadata.id" --raw-output
qa_import_id = result.s
print("The new import ID is: {}".format(qa_import_id))

The new import ID is: d7781b9b-706f-4f84-9fc2-37ba93f465f5


In [121]:
! cpdctl asset import get --space-id {qa_space_id} --import-id {qa_import_id}

...
              
ID:        d7781b9b-706f-4f84-9fc2-37ba93f465f5   
Created:   2021-02-22T06:43:20.629Z   
State:     completed   


List all assets in the QA space

In [224]:
! cpdctl asset search --space-id {qa_space_id} --query '*:*' --type-name asset

...
ID                                     Name                                                      Created                    Description   Type         State       Tags                               Size   
fd947b1e-f519-4e67-b937-ac6b652399a6   batch-job-outputs-connection                              2021-02-22T06:43:24.000Z                 connection   available   []                                 0   
0acc0587-2e30-4a24-8f25-e1cd35b53e13   car_rental_training_data.csv                              2021-02-22T06:43:25.000Z                 data_asset   available   [cpdctl-demo promoted-7fb76cf7]]   79518   
c996d65f-5afe-44a8-9549-48a54ddd0000   bank-marketing-batch-output.csv                           2021-02-22T06:43:25.000Z                 data_asset   available   [connected-data]                   0   
7372c4f5-a319-4242-973c-bb9ef695cf15   boston-house-prices-prediction-model                      2021-02-22T06:43:25.000Z                 wml_model    available   []            

### 3.3 Run batch deployment job in the QA space<a class="anchor" id="part3.3"></a>

Search for the imported script asset in the QA scpace

In [125]:
asset_type = 'script'
query = 'asset.name:{}'.format('batch_job_script.py')
jmes_query = 'results[0].metadata.asset_id'
result = ! cpdctl asset search --space-id {qa_space_id} --query '{query}' --type-name {asset_type} --output json --jmes-query "{jmes_query}" --raw-output
qa_script_id = result.s
print("ID of the script in QA space: {}".format(qa_script_id))


ID of the script in QA space: 87ac9f7c-ef61-427a-8ae6-50c6e0ca19b8


Create the script batch deployment in the QA space:

In [129]:
asset = {
    'id': qa_script_id
}
asset_json = json.dumps(asset)

hardware_spec = {
    'name': 'S'
}
hardware_spec_json = json.dumps(hardware_spec)

batch_json = '{}'

deployment_name = 'script_batch_deployment'

In [138]:
result = ! cpdctl ml deployment create --space-id {qa_space_id} --name '{deployment_name}' --asset '{asset_json}' --hardware-spec '{hardware_spec_json}' --batch '{batch_json}' --output json -j "metadata.id" --raw-output
qa_deployment_id = result.s
print("ID of the deployment in QA space: {}".format(qa_deployment_id))

ID of the deployment in QA space: 45fa001b-4202-43c9-b668-db2b3c7dc9a9


Create a deployment job

In [139]:
deployment_job_name = 'script_batch_deployment_job'

deployment = {
    'id': qa_deployment_id
}
deployment_json = json.dumps(deployment)

scoring = {
    "input_data_references": [
      {
        "type": "data_asset",
        "connection": {},
        "location": {
          "href": "/v2/assets/0acc0587-2e30-4a24-8f25-e1cd35b53e13?space_id=691dfe59-ffed-4241-a2a1-9c49be5aaf02"
        }
      }
    ],
    "output_data_reference": {
      "type": "data_asset",
      "connection": {},
      "location": {
        "href": "/v2/assets/c996d65f-5afe-44a8-9549-48a54ddd0000?space_id=691dfe59-ffed-4241-a2a1-9c49be5aaf02"
      }
    }
}
scoring_json = json.dumps(scoring)

In [140]:
result = ! cpdctl ml deployment-job create --space-id {qa_space_id} --name '{deployment_job_name}' --deployment '{deployment_json}' --scoring '{scoring_json}' --output json
qa_deployment_job = json.loads(result.s)
print(json.dumps(qa_deployment_job, indent=2))
qa_job_id = qa_deployment_job['entity']['platform_job']['job_id']
qa_run_id = qa_deployment_job['entity']['platform_job']['run_id']

{
  "entity": {
    "deployment": {
      "id": "45fa001b-4202-43c9-b668-db2b3c7dc9a9"
    },
    "platform_job": {
      "job_id": "500ddd7e-29a5-47b1-b802-af43c85ed694",
      "run_id": "850f5955-79a5-42fc-ace6-f315bcfb48be"
    },
    "scoring": {
      "input_data_references": [
        {
          "connection": {},
          "location": {
            "href": "/v2/assets/0acc0587-2e30-4a24-8f25-e1cd35b53e13?space_id=691dfe59-ffed-4241-a2a1-9c49be5aaf02"
          },
          "type": "data_asset"
        }
      ],
      "output_data_reference": {
        "connection": {},
        "location": {
          "href": "/v2/assets/c996d65f-5afe-44a8-9549-48a54ddd0000?space_id=691dfe59-ffed-4241-a2a1-9c49be5aaf02"
        },
        "type": "data_asset"
      },
      "status": {
        "state": "queued"
      }
    }
  },
  "metadata": {
    "created_at": "2021-02-22T07:20:21.087Z",
    "id": "1c26fb04-04ea-4112-a307-550c432a9f29",
    "name": "script_batch_deployment_job",
    "space_id

Wait for job completion

In [141]:
! cpdctl job run wait --job-id {qa_job_id} --run-id {qa_run_id} --space-id {qa_space_id}

...
                  
ID:            850f5955-79a5-42fc-ace6-f315bcfb48be   
Name:          job run   
Created:       2021-02-22T07:20:21Z   
Description:      
State:         Completed   
Tags:          []   


You can see the batch deployment log:

In [142]:
! cpdctl job run logs --job-id {qa_job_id} --run-id {qa_run_id} --space-id {qa_space_id}

...
total_count   results   
36            {   
36              "deployment": {   
36                "id": "45fa001b-4202-43c9-b668-db2b3c7dc9a9"   
36              },   
36              "platform_job": {   
36                "job_id": "500ddd7e-29a5-47b1-b802-af43c85ed694",   
36                "run_id": "850f5955-79a5-42fc-ace6-f315bcfb48be"   
36              },   
36              "scoring": {   
36                "input_data_references": [   
36                  {   
36                    "connection": {},   
36                    "location": {   
36                      "href": "/v2/assets/0acc0587-2e30-4a24-8f25-e1cd35b53e13?space_id=691dfe59-ffed-4241-a2a1-9c49be5aaf02"   
36                    },   
36                    "type": "data_asset"   
36                  }   
36                ],   
36                "output_data_reference": {   
36                  "connection": {},   
36                  "location": {   
36                    "href": "/v2/assets/c996d65f-5afe-44a8-9

## 4. Demo 3: Download the script from the QA space and upload it to the production space <a class="anchor" id="part4"></a>

Before starting with this section, please ensure that you have run the cells in all previous sections: [Section 1](#part1), [Section 2](#part2) and [Section 3](#part3).

### 4.1 Downloading a script <a class="anchor" id="part4.1"></a>

Download the script from the QA space

In [148]:
jmes_query = 'attachments[0].object_key'
result = ! cpdctl asset get --space-id {qa_space_id} --asset-id {qa_script_id} --output json --jmes-query {jmes_query} --raw-output
qa_script_path = result.s
print('Path to the QA script: {}'.format(qa_script_path))

Path to the QA script: script/batch_job_script_63g58m853sl56suvhibtt2c99.py


In [150]:
local_script_path = "qa_batch_job_script.py"
! cpdctl asset file download --space-id {qa_space_id} --path {qa_script_path} --output-file {local_script_path}

...
OK
Output written to qa_batch_job_script.py


In [151]:
! head {local_script_path}

import os
import ibm_boto3
import json
import pandas as pd
import requests
from botocore.client import Config
from ibm_watson_machine_learning import APIClient


### Function to get asset details using REST API. This won't be needed once python client adds attachment details in asset meta ###


### 4.2 Creating the new script in production space<a class="anchor" id="part4.2"></a>

Upload the script file to production space:

In [167]:
prod_space_name = 'cpdctl-prod-space-for-scripts'
jmes_query = "resources[?entity.name == '{}'] | [0].metadata.id".format(prod_space_name)
result = ! cpdctl space list --output json --jmes-query "{jmes_query}" --raw-output
prod_space_id = result.s
print('Production space ID: {}'.format(prod_space_id))

Production space ID: 6c205951-1c61-49b9-b46d-e5e199492775


In [168]:
remote_script_path = "script/batch_job_script.py"
! cpdctl asset file upload --path {remote_script_path} --file {local_script_path} --space-id {prod_space_id}

...
OK


In [169]:
jmes_query = "resources[0].metadata.asset_id"
result = ! cpdctl environment software-specification list --space-id {prod_space_id} --name '{software_specification_name}' --output json --jmes-query '{jmes_query}' --raw-output
prod_software_specification_id = result.s
print("software specification id: {}".format(prod_software_specification_id))

software specification id: e4429883-c883-42b6-87a8-f419d64088cd


Specify the metadata, entity and attachments of the script file in the production space:

In [178]:
script_ts = datetime.now().strftime('%Y%m%d-%H%M%S')

metadata = {
    "name": "batch_job_script_{}".format(script_ts),
    "asset_type": "script",
    "asset_category": "USER",
    "origin_country": "us"
}
metadata_json = json.dumps(metadata)

entity = {
    "script": {
        "language": {
            "name": "python3"
        },
        "software_spec": {
            "base_id": "{}".format(prod_software_specification_id),
            "name": software_specification_name
        }
    }
}
entity_json = json.dumps(entity)

attachments = [
    {
        "asset_type": "script",
        "name": "batch_job_script.py",
        "description": "attachment for script",
        "mime": "application/text",
        "object_key": remote_script_path
    }
]
attachments_json = json.dumps(attachments)

Create a Python script asset:

In [179]:
result = ! cpdctl asset create --metadata '{metadata_json}' --entity '{entity_json}' --attachments '{attachments_json}' --space-id {prod_space_id} --output json -j "metadata.asset_id" --raw-output
prod_script_id = result.s
print("ID of the script in production space: {}".format(prod_script_id))

ID of the script in production space: d42e3972-bcff-487e-bbb9-2058211af83a


### 4.3 Updating the existing script deployment<a class="anchor" id="part4.3"></a>

Update the **existing** script batch deployment in the production space:

In [186]:
deployment_name = 'script_batch_deployment'
jmes_query = 'resources[0].metadata.id'
result = ! cpdctl ml deployment list --space-id {prod_space_id} --name {deployment_name} --output json --jmes-query '{jmes_query}' --raw-output
prod_deployment_id = result.s
print('Existing production deployment ID: {}'.format(prod_deployment_id))

Existing production deployment ID: 8be0d371-f2e8-4dd4-8c90-dd5fbd66800d


Update the deployed asset with the newly created script

In [188]:
asset = {
    'id': prod_script_id
}
asset_json = json.dumps(asset)

In [189]:
! cpdctl ml deployment update --space-id {prod_space_id} --deployment-id {prod_deployment_id} --asset '{asset_json}'

...
              
ID:        8be0d371-f2e8-4dd4-8c90-dd5fbd66800d   
Name:      script_batch_deployment   
Created:   2021-02-22T08:08:21.866Z   
State:     ready   
Tags:      []   


Get the deployment job

In [194]:
jmes_query = 'resources[0].entity.platform_job.job_id'
result = ! cpdctl ml deployment-job list --deployment-id {prod_deployment_id} --space-id {prod_space_id} --output json --jmes-query '{jmes_query}' --raw-output
prod_job_id = result.s
print('Production job ID: {}'.format(prod_job_id))

Production job ID: c00b56a7-2eca-44d7-9d9e-100357591d7f


In [202]:
run = '{}'
jmes_query = 'metadata.asset_id'
result = ! cpdctl job run create --space-id {prod_space_id} --job-id {prod_job_id} --job-run '{run}' --output json --jmes-query '{jmes_query}' --raw-output
prod_run_id = result.s
print('ID of the job run in production space: {}'.format(prod_run_id))

ID of the job run in production space: 17f85b05-8670-452f-babb-9c4882a04fbc


Wait for job completion

In [203]:
! cpdctl job run wait --job-id {prod_job_id} --run-id {prod_run_id} --space-id {prod_space_id}

...
                  
ID:            17f85b05-8670-452f-babb-9c4882a04fbc   
Name:          job run   
Created:       2021-02-22T08:33:30Z   
Description:      
State:         Completed   
Tags:          []   


You can see the batch deployment log:

In [205]:
! cpdctl job run logs --job-id {prod_job_id} --run-id {prod_run_id} --space-id {prod_space_id}

...
total_count   results   
39            {   
39              "deployment": {   
39                "id": "8be0d371-f2e8-4dd4-8c90-dd5fbd66800d"   
39              },   
39              "hardware_spec": {   
39                "id": "f3ebac7d-0a75-410c-8b48-a931428cc4c5"   
39              },   
39              "platform_job": {   
39                "job_id": "c00b56a7-2eca-44d7-9d9e-100357591d7f",   
39                "run_id": "17f85b05-8670-452f-babb-9c4882a04fbc"   
39              },   
39              "scoring": {   
39                "input_data_references": [   
39                  {   
39                    "connection": {},   
39                    "location": {   
39                      "href": "/v2/assets/8dbf6071-8e0e-4bae-96f8-e5035c607eca?space_id=6c205951-1c61-49b9-b46d-e5e199492775"   
39                    },   
39                    "type": "data_asset"   
39                  }   
39                ],   
39                "output_data_reference": {   
39            

## 5. Cleanup <a class="anchor" id="part5"></a>

Delete QA space

In [227]:
! cpdctl space delete --space-id {qa_space_id}

...
OK


### Author

Rafał Bigaj, System Architect with long successful record of building and leading teams. Broad and practical knowledge in the area of cloud computing, machine learning and distributed systems development. 

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.